In [0]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from scipy.io import loadmat
import pickle

#create matrix

In [0]:
def make_mat(size,div_rate,test_size):
  ran_mat = np.random.randint(6,size = (size,size))
  u,s,vh = np.linalg.svd(ran_mat)
  sigma = np.zeros((ran_mat.shape[0], ran_mat.shape[1]))
  sigma[:ran_mat.shape[0]//div_rate, :ran_mat.shape[0]//div_rate] = np.diag(s)[:ran_mat.shape[0]//div_rate, :ran_mat.shape[0]//div_rate]
  traun_mat = u.dot(sigma).dot(vh)
  
  mask = np.random.permutation(range(size**2))[-int(size**2*test_size):]
  
  mask_xy = []
  for idx in range(len(mask)):
    x = mask[idx]//size
    y = mask[idx] % size
    mask_xy.append((x,y))
  train = np.copy(traun_mat) 
  for (x,y) in mask_xy:
    train[x,y] = 0
  label = [traun_mat[i] for i in mask_xy]
  return ran_mat,mask_xy,train,label

In [0]:
class data_obj():
  def __init__ (self, ur_size, it_size,div, test_size):
    self.ur_size = ur_size
    self.it_size = it_size
    self.div = div
    self.test_size = test_size
  def make_ran_mat (self):
    self.ran_mat, self.mask_xy, self.train, self.label = make_mat(self.ur_size,self.div,self.test_size)

#naive distance

In [0]:
def cor_nan(a,b):
  new_a = []
  new_b = []
  for i in range(len(a)):
    if a[i] > 0 and b[i] > 0:
      new_a.append(a[i])
      new_b.append(b[i])
  if len(new_a) <2:
    return 0
  else:
    return pearsonr(new_a,new_b)[0]

In [0]:
def make_sims(rat,axis):
  sims = np.zeros((rat.shape[axis],rat.shape[axis]))
  if axis == 1:
    rat = rat.transpose()
  for i in range (len(sims)):
    for j in range(i):
      sims[i,j] = cor_nan(rat[i],rat[j])
      sims[j,i] = cor_nan(rat[i],rat[j])
  return sims

#neural distance

In [0]:
def to_str(l):
  for i in range(len(l)):
    l[i] = str(l[i])
  return l
def item_vec(train,axis):
  split_list = []
  if axis == 1:
    train = train.transpose()
  for i in range (train.shape[0]):
    l = np.where(train[i]>4)[0].tolist()
    split_list.append(to_str(l))
    index1 =  np.where(train[i]<=4)[0]
    index2 =  np.where(train[i]>0)[0]
    intersec = np.intersect1d(index1,index2).tolist()
    s2 = to_str(intersec)
                       
    split_list.append(s2)
  
  return split_list

In [0]:
for i in range(data.train.shape[0]):
  l = np.where(data.train[i]>4)[0].tolist()
  print(to_str(l))

['0', '5', '8', '11', '12', '13', '14', '15', '18', '31', '41', '43', '44', '47', '49', '54', '56', '58', '80', '81', '85', '86', '88', '92', '99', '107', '108', '110', '112', '113', '114', '123', '126', '128', '136', '149', '151', '153', '164', '165', '167', '168', '169', '170', '171', '172', '173', '174', '175', '176', '177', '180', '182', '189', '190', '194', '195', '197', '201', '203', '206', '207', '215', '220', '223', '227', '234', '241', '245', '252', '257', '267', '268', '269']
['49', '99', '126', '241', '250', '271', '274', '284', '301', '310', '312', '315']
['319', '320', '327', '339', '345', '346']
['257', '293', '299', '300', '302', '326', '328', '353', '358', '359', '361']
['41', '88', '99', '100', '108', '152', '162', '173', '180', '185', '188', '227', '256', '381', '389', '395', '400', '407', '427', '429', '432', '435']
['13', '58', '97', '99', '123', '126', '130', '131', '134', '135', '164', '172', '184', '196', '210', '215', '237', '461', '468', '473', '474', '478', '4

In [0]:
from gensim.models import Word2Vec
import datetime
def get_vec_sims(split_list,epoch,size,mat_size):
  model = Word2Vec(sentences = split_list, # We will supply the pre-processed list of moive lists to this parameter
                 iter = epoch, # epoch
                 min_count = 5, # a movie has to appear more than 5 times to be keeped
                 size = size, # size of the hidden layer
                 workers = 4, # specify the number of threads to be used for training
                 sg = 1, # Defines the training algorithm. We will use skip-gram so 1 is chosen.
                 hs = 0, # Set to 0, as we are applying negative sampling.
                 negative = 5, # If > 0, negative sampling will be used. We will use a value of 5.
                 window = 9999999)
  wv_sims = np.zeros((mat_size,mat_size))
  item_vec = model.wv
  for i in range(mat_size):
    if str(i) in item_vec.vocab:
      for j in range(i):
        if str(j) in item_vec.vocab:
          wv_sims[i,j] = item_vec.similarity(str(i),str(j))
          wv_sims[j,i] = item_vec.similarity(str(i),str(j))  
  return wv_sims

In [0]:
epoch = 5
size = 200

#knn 

In [0]:
def knn_ur(ur_id,mv_id,train,sims,n):
  targ = sims[ur_id]
  targ_sort = np.argsort(targ)
  count = 0
  rat = 0
  start = 0
  sims_sum = 0
  while count < n:
    idx = np.where(targ_sort == start)[0][0]
    if train[idx,mv_id] >= 0:     
      rat += train[idx,mv_id] * sims[ur_id,idx]
      count += 1
      start += 1
      sims_sum += sims[ur_id,idx]
    #print(ur_id,idx,train_mat[ur_id,idx],start)
    else:
      start += 1
  result = rat/sims_sum
  if result < 1:
    result = 1
  if result > 5:
    result = 5
  return result

In [0]:
def knn_mv(ur_id,mv_id,train,sims,n):
  targ = sims[mv_id]
  targ_sort = np.argsort(-targ)
  count = 0
  rat = 0
  start = 0
  sims_sum = 0
  while count < n:
    idx = np.where(targ_sort == start)[0][0]
    if train[ur_id,idx] >= 0:
      #print(idx)
     # print(train[ur_id,idx],sims[mv_id,idx])
      rat += train[ur_id,idx] * sims[mv_id,idx]
     # print(rat)
      count += 1
      start += 1  
      sims_sum += sims[mv_id,idx]
     # print(sims_sum)
    else:
      start += 1
  result = rat/sims_sum
  if result < 1:
    result = 1
  if result > 5:
    result = 5
  return result

In [0]:
def neigh_list(sim,rating):
  #particular similarity and rating for a pair
  neigh = []
  for i in range(len(sims)):
    if rating[i] > 0:
      neigh.append(i)
  return neigh

In [0]:
def knn(sim,rating,neigh,n):
  index = np.argsort(sims[neigh])
  result = np.sum([sims[index][i] * rating[index][i] for i in range(n)])
  weight = np.sum([sims[index][i] for i in range(n)])
  return result/wight

In [0]:
def get_result(data_obj,n):
  it_result = [[] for i in range(n-2)]
  ur_result = [[] for i in range(n-2)]
  for i in range(2,n):
    for (x,y) in data_obj.mask_xy:
      it_result[i-2].append(knn_mv(x,y,data_obj.train,data_obj.it_sims,i))
    for (x,y) in data_obj.mask_xy:
      ur_result[i-2].append(knn_ur(x,y,data_obj.train,data_obj.ur_sims,i))
  data_obj.ur_result = ur_result
  data_obj.it_result = it_result
  

In [0]:
def get_neu_result(data_obj,n):
  it_neu_result = [[] for i in range(n-2)]
  ur_neu_result = [[] for i in range(n-2)]
  for i in range(2,n):
    for (x,y) in data_obj.mask_xy:
      it_neu_result[i-2].append(knn_mv(x,y,data_obj.train,data_obj.it_neu_sims,i))
    for (x,y) in data_obj.mask_xy:
      ur_neu_result[i-2].append(knn_ur(x,y,data_obj.train,data_obj.ur_neu_sims,i))
  data_obj.ur_neu_result = ur_neu_result
  data_obj.it_neu_result = it_neu_result
  

#graph signal processing

In [0]:
def find_nn(group,target,sims,n,limit):
  rank = np.argsort(-sims[target])
  count = 0
  neigh = []
  search = 0
  while count < n:
    if np.where(rank == search)[0][0] in group:
      neigh.append(rank[search])
      search +=1
      count +=1
    else:
      search +=1
    if search == limit:
      break
  return neigh

In [0]:
def item_net(item,tar_user,ur_sims,train,n,limit):
  network = np.zeros(ur_sims.shape)
  rated_user = np.where(train[:,item]>0)[0]
  for user in tar_user:
    neighs = find_nn(rated_user,user,ur_sims,n,limit)
    user_sum = 0
    for neigh in neighs:
      user_sum += ur_sims[user,neigh]
    for neigh in neighs:
      network[user,neigh] = ur_sims[user,neigh]/user_sum
  return network


In [0]:
def user_net(user,tar_item,it_sims,train,n,limit):
  rated_item = np.where(train[user]>0)[0]
  network = np.zeros(it_sims.shape)
  for item in tar_item:
    neighs = find_nn(rated_item,item,it_sims,n,limit)
    item_sum = 0
    for neigh in neighs:
      item_sum += it_sims[item,neigh]
    for neigh in neighs:
      network[item,neigh] = it_sims[item,neigh]/item_sum
  return network

In [0]:
def network_feed_it(data, n, limit, iter):
  result = np.zeros(len(data.label))
  for item in range(data.it_size):
    tar = []
    idx = []
    for mask in range(len(data.mask_xy)):
      if data.mask_xy[mask][1] == item:
        tar.append(data.mask_xy[mask][0])
        idx.append(mask)
    F = item_net(item,tar,data.ur_sims,data.train,n,limit)
    for ite in range(iter-1):
      F = F.dot(F)
    x_hat = F.dot(data.train[:,item])[tar]
    for i in range(len(x_hat)):
      result[idx[i]] = x_hat[i]
      
  for pred in range(len(result)):
    if result[pred] < 1:
      result[pred] = 1
    if result[pred] > 5:
      result[pred] = 5
  return result

In [0]:
def network_feed_ur(data, n, limit, iter):
  result = np.zeros(len(data.label))
  for user in range(data.ur_size):
    tar = []
    idx = []
    for mask in range(len(data.mask_xy)):
      if data.mask_xy[mask][0] == user:
        tar.append(data.mask_xy[mask][1])
        idx.append(mask)
    F = user_net(user,tar,data.ur_sims,data.train,n,limit)
    for ite in range(iter-1):
      F = F.dot(F)
    x_hat = F.dot(data.train[user])[tar]
    for i in range(len(x_hat)):
      result[idx[i]] = x_hat[i]
  for pred in range(len(result)):
    if result[pred] < 1:
      result[pred] = 1
    if result[pred] > 5:
      result[pred] = 5
  return result

In [0]:
def test(data_obj,limit):
  it_result = {}
  ur_result = {}
  for neigh in range(2,8):
    for iter in range(1,3):
      result = network_feed_it(data_obj,neigh,limit,iter)
      it_result[(neigh,iter)] = result
  data_obj.gsp_it_result = it_result
 # for neigh in range(2,8):
 #   for iter in range(1,8):
 #     result = network_feed_ur(data_obj,neigh,limit,iter)
 #     ur_result[(neigh,iter)] = result
 # data_obj.gsp_ur_result = ur_result

#metric

In [0]:
def RMSE(pred,label):
  
  valid = np.where(pred>0)[0]
  return ((np.sum((pred[valid]-label[valid])**2))/len(pred[valid]))**(1/2)

In [0]:
def error_rate(pred,label):
  error = 0
  for i in range(len(pred)):
    error += (abs((pred[i]-label[i])/pred[i]))
  return error/len(pred)

In [0]:
def fob_error(pred,rat_mat,mask):
  recon_mat = np.copy(rat_mat)
  for i in range(len(pred)):
    if pred[i] >0:
      rat_mat[mask[i]] = pred[i]
  return np.linalg.norm(np.abs(rat_mat - recon_mat),ord='fro')


In [0]:
class metric():
  def __init__(self,pred,label,rat_mat,mask):
    self.pred = np.array(pred)[:10000]
    self.label = np.array(label)
    self.rat_mat = rat_mat
    self.mask = mask
  def compute_error(self):
    self.rmse = RMSE(self.pred,self.label)
    #self.error_rate = error_rate(self.pred,self.label)
    self.fob_error = fob_error(self.pred,self.rat_mat,self.mask)

In [0]:
def get_eval(data):
  data.it_met = []
  for i in range(len(data.it_result)):
    met = metric(data.it_result[i],data.label,data.ran_mat,data.mask_xy)
    met.compute_error()
    data.it_met.append(met.__dict__)
    
  data.ur_met = []
  for i in range(len(data.ur_result)):
    met = metric(data.ur_result[i],data.label,data.ran_mat,data.mask_xy)
    met.compute_error()
    data.ur_met.append(met.__dict__)  

In [0]:
def get_neu_eval(data):
  data.it_neu_met = []
  for i in range(len(data.it_result)):
    met = metric(data.it_neu_result[i],data.label,data.ran_mat,data.mask_xy)
    met.compute_error()
    data.it_neu_met.append(met.__dict__)
    
  data.ur_neu_met = []
  for i in range(len(data.ur_result)):
    met = metric(data.ur_neu_result[i],data.label,data.ran_mat,data.mask_xy)
    met.compute_error()
    data.ur_neu_met.append(met.__dict__) 

In [0]:
def get_gsp_eval(data):
  data.it_gsp_met = {}
  for key in data.gsp_it_result:
    met = metric(data.gsp_it_result[key],data.label,data.ran_mat,data.mask_xy)
    met.compute_error()
    data.it_gsp_met[key] = met.__dict__

#Testing


In [0]:
size = 100
div_rat = [10,20,30,40]
test_siz = [0.1,0.2,0.3]

In [0]:
#create random matrix
data_obj_dict = {}
for i in range(len(div_rat)):
  for j in range(len(test_siz)):
    data_obj_dict[(div_rat[i],test_siz[j])] = data_obj(size,size,div_rat[i],test_siz[j])
    data_obj_dict[(div_rat[i],test_siz[j])].make_ran_mat()

In [0]:
test_sample = 0
for data_obj in data_dict.keys():
  start = datetime.datetime.now()
  data_dict[data_obj].ur_split_list = item_vec(data_dict[data_obj].train,0,4)
  data_dict[data_obj].ur_neu_sims = get_vec_sims(data_dict[data_obj].ur_split_list,epoch,size,data_dict[data_obj].ur_size)
  data_dict[data_obj].it_split_list = item_vec(data_dict[data_obj].train,1,4)
  data_dict[data_obj].it_neu_sims = get_vec_sims(data_dict[data_obj].it_split_list,epoch,size,data_dict[data_obj].it_size)
  print("test_sample: " + str(test_sample) + " Time passed: " + str(datetime.datetime.now()-start))
  test_sample +=1

In [0]:
for data_obj in data_obj_dict.keys():
  data_obj_dict[data_obj].ur_sims = make_sims(data_obj_dict[data_obj].train,0)
  data_obj_dict[data_obj].it_sims = make_sims(data_obj_dict[data_obj].train,1)

In [0]:
for data_obj in data_obj_dict.keys():
  get_result(data_obj_dict[data_obj],8)

In [0]:
for data_obj in data_obj_dict.keys():
  test(data_obj_dict[d ata_obj],99)

In [0]:
for key in data_dict.keys():
  get_eval(data_dict[key])


In [0]:
for key in data_dict.keys():
  get_neu_eval(data_dict[key])

In [0]:
for key in data_dict.keys():
  get_nn_result(data_dict[key],8)

In [0]:
for key in data_dict.keys():
  get_neu_result(data_dict[key],8)

In [0]:
trail = 1
for key in data_dict.keys():
  print('trail ', trail, 'trauncate rate ', data_dict[key].div, 'split_size ',data_dict[key].test_size )
  for i in range(len(data_dict[key].ur_met)):
    print('uers neigh = ', i+2, 'RMSE = ', data_dict[key].ur_met[i]['rmse'])
  for i in range(len(data_dict[key].ur_met)):
    print('item neigh = ', i+2, 'RMSE = ', data_dict[key].it_met[i]['rmse'])
  trail += 1

In [0]:
plot_list = np.zeros((12,6))
j = 0
tag = []
for key in data_dict.keys():
  for i in range(len(data_dict[key].ur_met)):
    plot_list[j,i] = data_dict[key].ur_met[i]['rmse']
  tag.append((data_dict[key].div,data_dict[key].test_size))
  j+=1  

In [0]:
tag = []
for i in ["10%","5%",'3%',"2.5%"]:
  for j in ['10%','20%','30%']:
    tag.append(i+' sigular value, '+j+' test set')

In [0]:
plot_list = np.zeros((12,6))
j = 0
#tag = []
for key in data_dict.keys():
  for i in range(len(data_dict[key].ur_met)):
    plot_list[j,i] = data_dict[key].ur_met[i]['rmse']
  #tag.append((data_dict[key].div,data_dict[key].test_size))
  j+=1  

In [0]:
plt.figure(figsize=(10,8))
for i in range(len(plot_list)):
  plt.plot(range(2,8),  plot_list[i], label = str(tag[i]))
plt.legend(bbox_to_anchor=(1, 1),prop={'size': 15})
plt.show()


#movielen

In [0]:
rat = pd.read_csv('drive/My Drive/Graph Signal Processing/ratings.csv')

In [0]:
rat.head()

,userId,movieId,rating,comment
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [0]:
U = max(rat['userId']) 
I = max(rat['movieId']) 
print(U,I)

943 1682


In [0]:
rat_mat = np.zeros((U,I))
for entry in range(len(rat)):
  rat_mat[rat.iloc[entry]['userId']-1,rat.iloc[entry]['movieId']-1] = rat.iloc[entry]['rating']

In [0]:
mask = np.random.permutation(len(rat))[:len(rat)//10]

In [0]:
mask_xy = []
label = []
train = np.copy(rat_mat)
for entry in mask:
  mask_xy.append((rat.iloc[entry]['userId']-1,rat.iloc[entry]['movieId']-1))
  train[rat.iloc[entry]['userId']-1,rat.iloc[entry]['movieId']-1] = 0
  label = rat.iloc[entry]['rating']

In [0]:
data = data_obj(U,I,None,None)
data.train = train
data.mask_xy = mask_xy
data.label = label
data.ran_mat = rat_mat

In [0]:
data.ur_sims = make_sims(data.train,0)
data.it_sims = make_sims(data.train,1)

/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [0]:
get_result(data,8)

In [0]:
get_eval(data)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in greater
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:

data.ur_split_list = item_vec(data.train,0)
data.ur_neu_sims = get_vec_sims(data.ur_split_list,epoch,size,data.ur_size)
data.it_split_list = item_vec(data.train,1)
data.it_neu_sims = get_vec_sims(data.it_split_list,5,size,data.it_size)


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
get_neu_result(data,8)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars


In [0]:
get_neu_result(data,8)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars


In [0]:
get_neu_eval(data)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in greater
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
plt.figure(figsize=(10,8))
plt.plot(range(2,8),[data.it_met[i]['rmse'] for i in range(6)])
plt.plot(range(2,8),[data.it_neu_met[i]['rmse'] for i in range(6)])
plt.legend(['Pearson R','Negative Sampling'])

In [0]:
get_gsp_eval(data)